In [ ]:
! pip install langchain_openai langchain_community langchain_core -U

In [ ]:
from langchain_core.tools import tool
from google.colab import userdata
from langchain_openai import ChatOpenAI

@tool
def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="qwen2-72b-instruct", temperature=0)

tool = llm.bind_tools([amultiply])

result = tool.invoke('a 为 3 b 是 6')

if hasattr(result, 'tool_calls') and result.tool_calls:
    print(result.tool_calls[0]['args'])
    print(type(result.tool_calls[0]['args']))
    
    print(amultiply.invoke(result.tool_calls[0]['args']))
else:
    print(result.content)





In [12]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langchain_core.tools import tool
from google.colab import userdata
from langchain_openai import ChatOpenAI

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

    
idl1 = """
message ToolRequest {
    string tool_id = 1;  // 工具的唯一标识符
    int32 version = 2;   // 工具的版本号
}
"""

idl2 = """
message ToolRequest {
    string tool_name = 1;  // 工具的名称
    int32 version = 2;     // 工具的版本号
}
"""

idl3 = """
message ToolRequest {
    string tool_category = 1;  // 工具的类别
    int32 version = 2;         // 工具的版本号
}
"""

idl4 = """
message ToolRequest {
    string tool_description = 1;  // 工具的描述信息
    int32 version = 2;            // 工具的版本号
}
"""

class InterfaceDefinition(TypedDict):
    path: str
    description: str

class PSMDefinition(TypedDict):
    name: str
    interfaces: List[InterfaceDefinition]
    

interfaceMap: List[PSMDefinition] = [
    {
        "name": "anchor.psm1",
        "interfaces": [
            {
                "path": "/ark/anchor1",
                "idl": idl1
            },
            {
                "path": "/ark/anchor2",
                "idl": idl2
            },
        ],
    },
    {
        "name": "anchor.psm2",
        "interfaces": [
            {
                "path": "/webcast/tool1",
                "idl": idl3
            },
            {
                "path": "/webcast/tool2",
                "idl": idl4
            },
        ],
    },
]

def find_idl_by_name_and_path(name, path):
    for psm in interfaceMap:
        if psm["name"] == name:
            for interface in psm["interfaces"]:
                if interface["path"] == path:
                    return interface["idl"]

class ConfigDict(TypedDict):
    path: Annotated[str, "接口地址"]
    psm: Annotated[int, "接口对应的psm"]

@tool
def explain_key_by_path(
    config: ConfigDict,  # 定义一个包含 path 和 psm 的字典
    keyName: Annotated[str, "字段名称"],
):
    """字段解释工具，通过接口地址和接口对应的psm查找接口定义，并根据字段名来获取字段说明"""
    # 你可以选择不使用 config 参数
    
    idl = find_idl_by_name_and_path(**config)
    
    return idl

print(explain_key_by_path.args_schema.schema())

llm2 = ChatOpenAI(model="qwen2-72b-instruct", temperature=0)

llm_tool = llm2.bind_tools([explain_key_by_path])

result = tool.invoke('a 为 1 b 是 2')

if hasattr(result, 'tool_calls') and result.tool_calls:
    print(result.tool_calls[0]['args'])
    print(type(result.tool_calls[0]['args']))
    
    print(amultiply.invoke(result.tool_calls[0]['args']))
else:
    print(result.content)



# # 示例使用
# name = "anchor.psm1"
# path = "/ark/anchor2"
# idl = find_idl_by_name_and_path(name, path)
# print(idl)  # 输出: idl1




{'title': 'explain_key_by_pathSchema', 'description': '字段解释工具，通过接口地址和接口对应的psm查找接口定义，并根据字段名来获取字段说明', 'type': 'object', 'properties': {'config': {'$ref': '#/definitions/ConfigDict'}, 'keyName': {'title': 'Keyname', 'description': '字段名称', 'type': 'string'}}, 'required': ['config', 'keyName'], 'definitions': {'ConfigDict': {'title': 'ConfigDict', 'type': 'object', 'properties': {'path': {'title': 'Path', 'type': 'string'}, 'psm': {'title': 'Psm', 'type': 'integer'}}, 'required': ['path', 'psm']}}}
